# Report Generation Tutorial

PyGSTi is able to construct polished report documents, which provide high-level summaries as well as detailed analyses of results (Gate Set Tomography (GST) and model-testing results in particular).  Reports are intended to be quick and easy way of analyzing `Model`-type estimates, and pyGSTi's report generation functions are specifically designed to interact with the `ModelEstimateResults` object (producted by several high-level algorithm functions - see, for example, the [GST overview tutorial](../algorithms/GST-Overview.ipynb) and [GST functions tutorial](../algorithms/GST-Drivers.ipynb).).  The report generation functions in pyGSTi takes one or more results (often `ModelEstimateResults`-type) objects as input and produces an HTML file as output.  The HTML format allows the reports to include **interactive plots** and **switches** (see the [workspace switchboard tutorial](advanced/WorkspaceSwitchboards.ipynb), making it easy to compare different types of analysis or data sets.  

PyGSTi's reports are stand-alone HTML documents which cannot run Python.  Thus, all the results displayed in a report must be pre-computed (in Python).  If you find yourself wanting to fiddle with things and feel that these reports are too static, please consider using a `Workspace` object (see the [Workspace tutorial](Workspace.ipynb)) within a Jupyter notebook, where you can intermix report tables/plots and Python.  Internally, functions like `construct_standard_report` (see below) are simple factories for `Report` objects, which are in turn little more than a wrapper around a `Workspace` object and a set of instructions for how to generate reports of different output formats.


### Get some `ModelEstimateResults`
We start by performing GST using `do_long_sequence_gst`, as usual, to create a `ModelEstimateResults` object (we could also have just loaded one from file).  See the [GST functions tutorial](../algorithms/GST-Driverfunctions.ipynb) for more details.

In [1]:
import pygsti
from pygsti.modelpacks import smq1Q_XYI

target_model = smq1Q_XYI.target_model()
prep_fiducials = smq1Q_XYI.prep_fiducials()
meas_fiducials = smq1Q_XYI.meas_fiducials()
germs = smq1Q_XYI.germs()
maxLengths = [1,2,4,8,16]
ds = pygsti.io.load_dataset("../tutorial_files/Example_Dataset.txt", cache=True)

#Run GST
target_model.set_all_parameterizations("TP") #TP-constrained
results = pygsti.do_long_sequence_gst(ds, target_model, prep_fiducials, meas_fiducials, germs,
                                      maxLengths, verbosity=3)

Loading from cache file: ../tutorial_files/Example_Dataset.txt.cache
--- Circuit Creation ---
   616 sequences created
   Dataset has 1120 entries: 616 utilized, 0 requested sequences were missing
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.243708948119547
  1.1796799227093193
  0.9631622821716201
  0.9415276081969872
  0.04762148814438346
  0.015427853978163288
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
--- Iterative MLGST: Iter 1 of 5  92 operation sequences ---: 
  --- Minimum Chi^2 GST ---
    bulk_evaltree: created initial tree (92 strs) in 0s
    bulk_evaltree: split tree (1 subtrees) in 0s
    Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
     groups of ~1 procs each, to distribute over 43 params (taken as 1 param groups of ~43 params).
    -

### Make a report
Now that we have `results`, we use the `construct_standard_report` method within `pygsti.report` to generate a `Report`.  
`pygsti.report.construct_standard_report` is the most commonly used report factory function in pyGSTi, as it is appropriate for smaller models (1- and 2-qubit) which have *operations that are or can be represeted as dense matrices and/or vectors*.  

Once constructed, a `Report` object can be used to write a report as an HTML document, PDF, or notebook.  To open a HTML-format report, you open the `main.html` file directly inside the report's folder.  Setting `auto_open=True` makes the finished report open in your web browser automatically.  

In [2]:
report = pygsti.report.construct_standard_report(results, title="GST Example Report", verbosity=1)
#HTML
report.write_html("../tutorial_files/exampleReport", auto_open=True, verbosity=1)

print("\n")

#PDF
report.write_pdf("../tutorial_files/exampleReport.pdf", auto_open=True, verbosity=1)

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI


Generating LaTeX source...


/usr/local/lib/python3.7/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning:

Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.

/usr/local/lib/python3.7/site-packages/matplotlib/colors.py:504: RuntimeWarning:

invalid value encountered in less



Compiling with `pdflatex -interaction=nonstopmode -halt-on-error -shell-escape`
Initial output PDF ../tutorial_files/exampleReport.pdf successfully generated.
Final output PDF ../tutorial_files/exampleReport.pdf successfully generated. Cleaning up .aux and .log files.
Opening ../tutorial_files/exampleReport.pdf...


There are several remarks about these reports worth noting:
1. The **HTML reports are the primary report type in pyGSTi**, and are much more flexible.  The PDF reports are more limited (they can only display a *single* estimate and gauge optimization), and essentially contain a subset of the information and descriptive text of a HTML report.  So, if you can, use the HTML reports.  The PDF report's strength is its portability: PDFs are easily displayed by many devices, and they embed all that they need neatly into a single file.  **If you need to generate a PDF report** from `Results` objects that have multiple estimates and/or gauge optimizations, consider using the `Results` object's `view` method to single out the estimate and gauge optimization you're after.
2. It's best to use **Firefox** when opening the HTML reports. (If there's a problem with your brower's capabilities it will be shown on the screen when you try to load the report.)
3. You'll need **`pdflatex`** on your system to compile PDF reports.
4. To familiarize yourself with the layout of an HTML report, click on the gray **"Help" link** on the black sidebar.

### Multiple estimates in a single report
Next, let's analyze the same data two different ways: with and without the TP-constraint (i.e. whether the gates *must* be trace-preserving) and furthermore gauge optmimize each case using several different SPAM-weights.  In each case we'll call `do_long_sequence_gst` with `gaugeOptParams=False`, so that no gauge optimization is done, and then perform several gauge optimizations separately and add these to the `Results` object via its `add_gaugeoptimized` function.

In [3]:
#Case1: TP-constrained GST
tpTarget = target_model.copy()
tpTarget.set_all_parameterizations("TP")
results_tp = pygsti.do_long_sequence_gst(ds, tpTarget, prep_fiducials, meas_fiducials, germs,
                                      maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_tp.estimates['default']
mdlFinal = est.models['final iteration estimate']
mdlTarget = est.models['target']
for spamWt in [1e-4,1e-2,1.0]:
    mdl = pygsti.gaugeopt_to_target(mdlFinal,mdlTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, mdl, "Spam %g" % spamWt)

--- Circuit Creation ---
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.243708948119547
  1.1796799227093193
  0.9631622821716201
  0.9415276081969872
  0.04762148814438346
  0.015427853978163288
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
--- Iterative MLGST: [##################################################] 100.0%  616 operation sequences ---
Iterative MLGST Total Time: 6.3s


In [4]:
#Case2: "Full" GST
fullTarget = target_model.copy()
fullTarget.set_all_parameterizations("full")
results_full = pygsti.do_long_sequence_gst(ds, fullTarget, prep_fiducials, meas_fiducials, germs,
                                           maxLengths, gaugeOptParams=False, verbosity=1)

#Gauge optimize
est = results_full.estimates['default']
mdlFinal = est.models['final iteration estimate']
mdlTarget = est.models['target']
for spamWt in [1e-4,1e-2,1.0]:
    mdl = pygsti.gaugeopt_to_target(mdlFinal,mdlTarget,{'gates':1, 'spam':spamWt})
    est.add_gaugeoptimized({'itemWeights': {'gates':1, 'spam':spamWt}}, mdl, "Spam %g" % spamWt)

--- Circuit Creation ---
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.243708948119547
  1.1796799227093193
  0.9631622821716201
  0.9415276081969872
  0.04762148814438346
  0.015427853978163288
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
--- Iterative MLGST: [##################################################] 100.0%  616 operation sequences ---
Iterative MLGST Total Time: 8.0s


We'll now call the *same* `construct_standard_report` function but this time instead of passing a single `Results` object as the first argument we'll pass a *dictionary* of them.  This will result in a **HTML report that includes switches** to select which case ("TP" or "Full") as well as which gauge optimization to display output quantities for.  PDF reports cannot support this interactivity, and so **if you try to generate a PDF report you'll get an error**.

In [5]:
ws = pygsti.report.Workspace()
report = pygsti.report.construct_standard_report(
    {'TP': results_tp, "Full": results_full}, title="Example Multi-Estimate Report", ws=ws, verbosity=2)
report.write_html("../tutorial_files/exampleMultiEstimateReport", auto_open=True, verbosity=2)

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1303: UserWarning:

PDF output is not available for reports with multiple datasets



Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.
Statistical hypothesis tests did NOT find inconsistency between the datasets at 5.00% significance.


In the above call we construct `ws` - a `Workspace` object.  PyGSTi's `Workspace` objects function as both a factory for figures and tables as well as a smart cache for computed values.  A `Workspace` object can optionally be passed as an argument to `construct_standard_report`, where it is used to create all figures in the report.  As an intended side effect, each of these figures is cached, along with some of the intermediate results used to create it.  By passing a preconstructed `Workspace` object to `construct_standard_report`, we allow it to utilize previously cached quantities.

**Another way**: Because both `results_tp` and `results_full` above used the same dataset and operation sequences, we could have combined them as two estimates in a single `ModelEstimateResults` object (see the previous tutorial on pyGSTi's results objects).  This can be done by renaming at least one of the `"default"`-named estimates in `results_tp` or `results_full` (below we rename both) and then adding the estimate within `results_full` to the estimates already contained in `results_tp`: 

In [6]:
results_tp.rename_estimate('default','TP')
results_full.rename_estimate('default','Full')
results_both = results_tp.copy() #copy just for neatness
results_both.add_estimates(results_full, estimatesToAdd=['Full'])

Creating a report using `results_both` will result in the same report we just generated.  We'll demonstrate this anyway, but in addition we'll supply `construct_standard_report` with the same `Workspace` we previously used to generate a report.  This tells the constructed `Report` to use any cached values contained in a given *input* `Workspace` to expedite report generation.  Since our workspace object has the exact quantities we need cached in it, you'll notice a significant speedup.  Finally, note that even though there's just a single `Results` object, you **still can't generate a PDF report** from it because it contains multiple estimates.

In [7]:
pygsti.report.construct_standard_report(
    results_both,
    title="Example Multi-Estimate Report (v2)", 
    ws=ws, verbosity=2
).write_html("../tutorial_files/exampleMultiEstimateReport2", auto_open=True, verbosity=2)

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1308: UserWarning:

PDF output is not available for reports with multiple gauge opts



### Multiple estimates and `do_stdpractice_gst`
It's no coincidence that a `Results` object containing multiple estimates using the same data is precisely what's returned from `do_stdpractice_gst` (see docstring for information on its arguments, and see the [GST functions tutorial](../algorithms/GST-Drivers.ipynb)).  This allows one to run GST multiple times, creating several different "standard" estimates and gauge optimizations, and plot them all in a single (HTML) report. 

In [8]:
results_std = pygsti.do_stdpractice_gst(ds, target_model, prep_fiducials, meas_fiducials, germs,
                                        maxLengths, verbosity=4, modes="TP,CPTP,Target",
                                        gaugeOptSuite=('stdgaugeopt','toggleValidSpam'))

# Generate a report with "TP", "CPTP", and "Target" estimates
pygsti.report.construct_standard_report(
    results_std, title="Post StdPractice Report", verbosity=1
).write_html("../tutorial_files/exampleStdReport", auto_open=True, verbosity=1)

--- Circuit Creation ---
   616 sequences created
   Dataset has 1120 entries: 616 utilized, 0 requested sequences were missing
-- Std Practice:  Iter 1 of 3  (TP) --: 
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.243708948119547
  1.1796799227093193
  0.9631622821716201
  0.9415276081969872
  0.04762148814438346
  0.015427853978163288
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
  --- Iterative MLGST: Iter 1 of 5  92 operation sequences ---: 
    --- Minimum Chi^2 GST ---
      bulk_evaltree: created initial tree (92 strs) in 0s
      bulk_evaltree: split tree (1 subtrees) in 0s
      Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
       groups of ~1 procs each, to distribute over 43 params (taken as 1 param groups of ~43 params).
      --- Outer Iter 

      bulk_evaltree: created initial tree (92 strs) in 0s
      bulk_evaltree: split tree (1 subtrees) in 0s
      Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
       groups of ~1 procs each, to distribute over 60 params (taken as 1 param groups of ~60 params).
      --- Outer Iter 0: norm_f = 1.10824e+07, mu=1, |x|=3.87298e-08, |J|=776.628
      --- Outer Iter 1: norm_f = 13896.2, mu=76.022, |x|=0.186556, |J|=1603.59
      --- Outer Iter 2: norm_f = 691.025, mu=25.3407, |x|=0.395611, |J|=805.502
      --- Outer Iter 3: norm_f = 305.58, mu=25.0676, |x|=0.576073, |J|=636.635
      --- Outer Iter 4: norm_f = 60.9768, mu=8.35586, |x|=0.488447, |J|=684.947
      --- Outer Iter 5: norm_f = 54.6428, mu=5.67173, |x|=0.481138, |J|=692.277
      --- Outer Iter 6: norm_f = 54.1427, mu=363.222, |x|=0.479168, |J|=693.573
      --- Outer Iter 7: norm_f = 53.9755, mu=743.052, |x|=0.478533, |J|=694.019
      --- Outer Iter 8: norm_f = 53.9567, mu=1177.28, 

      --- Outer Iter 98: norm_f = 53.5374, mu=223.108, |x|=0.484378, |J|=698.372
      --- Outer Iter 99: norm_f = 53.5359, mu=1715.96, |x|=0.484407, |J|=698.296
      --- Outer Iter 100: norm_f = 53.5356, mu=875.91, |x|=0.484441, |J|=698.302
      --- Outer Iter 101: norm_f = 53.535, mu=291.97, |x|=0.484509, |J|=698.323
      --- Outer Iter 102: norm_f = 53.5335, mu=97.3233, |x|=0.484706, |J|=698.382
      --- Outer Iter 103: norm_f = 53.531, mu=91.1406, |x|=0.485215, |J|=698.6
      --- Outer Iter 104: norm_f = 53.53, mu=734.775, |x|=0.485262, |J|=698.582
      --- Outer Iter 105: norm_f = 53.5295, mu=732.038, |x|=0.48532, |J|=698.591
      --- Outer Iter 106: norm_f = 53.5291, mu=567.125, |x|=0.485379, |J|=698.608
      --- Outer Iter 107: norm_f = 53.5287, mu=241.708, |x|=0.485455, |J|=698.633
      --- Outer Iter 108: norm_f = 53.5279, mu=168.223, |x|=0.485622, |J|=698.697
      --- Outer Iter 109: norm_f = 53.5276, mu=352.55, |x|=0.485733, |J|=698.761
      --- Outer Iter 110: no

      --- Outer Iter 1: norm_f = 569.98, mu=154.163, |x|=0.495192, |J|=2058.81
      --- Outer Iter 2: norm_f = 569.902, mu=128.153, |x|=0.494479, |J|=2060.69
      --- Outer Iter 3: norm_f = 569.886, mu=128.161, |x|=0.494422, |J|=2060.4
      --- Outer Iter 4: norm_f = 569.882, mu=1024.68, |x|=0.494355, |J|=2060.34
      --- Outer Iter 5: norm_f = 569.881, mu=806.892, |x|=0.494299, |J|=2060.27
      --- Outer Iter 6: norm_f = 569.881, mu=268.964, |x|=0.494234, |J|=2060.17
      --- Outer Iter 7: norm_f = 569.879, mu=106.881, |x|=0.494062, |J|=2059.92
      --- Outer Iter 8: norm_f = 569.878, mu=218.614, |x|=0.493879, |J|=2059.7
      --- Outer Iter 9: norm_f = 569.877, mu=591.508, |x|=0.493808, |J|=2059.62
      --- Outer Iter 10: norm_f = 569.876, mu=604.309, |x|=0.493753, |J|=2059.53
      --- Outer Iter 11: norm_f = 569.875, mu=604.344, |x|=0.493697, |J|=2059.46
      --- Outer Iter 12: norm_f = 569.875, mu=584.596, |x|=0.493643, |J|=2059.38
      Least squares message = Both actua

/Users/enielse/pyGSTi/pygsti/report/factory.py:1308: UserWarning:

PDF output is not available for reports with multiple gauge opts



### Reports with confidence regions
To display confidence intervals for reported quantities, you must do two things:

1. you must specify the `confidenceLevel` argument to `construct_standard_report`.
2. the estimate(s) being reported must have a valid confidence-region-factory.

Constructing a factory often means computing a Hessian, which can be time consuming, and so this is *not* done automatically.  Here we demonstrate how to construct a valid factory for the "Spam 0.001" gauge-optimization of the "CPTP" estimate by computing and then projecting the Hessian of the likelihood function. 

In [9]:
#Construct and initialize a "confidence region factory" for the CPTP estimate
crfact = results_std.estimates["CPTP"].add_confidence_region_factory('Spam 0.001', 'final')
crfact.compute_hessian(comm=None) #we could use more processors
crfact.project_hessian('intrinsic error')

pygsti.report.construct_standard_report(
    results_std, title="Post StdPractice Report (w/CIs on CPTP)",
    confidenceLevel=95, verbosity=1
).write_html("../tutorial_files/exampleStdReport2", auto_open=True, verbosity=1)

    
--- Hessian Projector Optimization from separate SPAM and Gate weighting ---
  Resulting intrinsic errors: 0.0104069 (gates), 0.00404055 (spam)
  Resulting sqrt(mean(operationCIs**2)): 0.013539
  Resulting sqrt(mean(spamCIs**2)): 0.00917909
Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1308: UserWarning:

PDF output is not available for reports with multiple gauge opts



### Reports with multiple *different* data sets
We've already seen above that `construct_standard_report` can be given a dictionary of `Results` objects instead of a single one.  This allows the creation of reports containing estimates for different `DataSet`s (each `Results` object only holds estimates for a single `DataSet`).  Furthermore, when the data sets have the same operation sequences, they will be compared within a tab of the HTML report.

Below, we generate a new data set with the same sequences as the one loaded at the beginning of this tutorial, proceed to run standard-practice GST on that dataset, and create a report of the results along with those of the original dataset.  Look at the **"Data Comparison" tab** within the gauge-invariant error metrics category.

In [10]:
#Make another dataset & estimates
depol_gateset = target_model.depolarize(op_noise=0.1)
datagen_gateset = depol_gateset.rotate((0.05,0,0.03))

#Compute the sequences needed to perform Long Sequence GST on 
# this Model with sequences up to lenth 512
circuit_list = pygsti.construction.make_lsgst_experiment_list(
    smq1Q_XYI.target_model(), smq1Q_XYI.prep_fiducials(), smq1Q_XYI.meas_fiducials(),
    smq1Q_XYI.germs(), [1,2,4,8,16,32,64,128,256,512])
ds2 = pygsti.construction.generate_fake_data(datagen_gateset, circuit_list, nSamples=1000,
                                             sampleError='binomial', seed=2018)
results_std2 = pygsti.do_stdpractice_gst(ds2, target_model, prep_fiducials, meas_fiducials, germs,
                                     maxLengths, verbosity=3, modes="TP,CPTP,Target",
                                     gaugeOptSuite=('stdgaugeopt','toggleValidSpam'))

pygsti.report.construct_standard_report(
    {'DS1': results_std, 'DS2': results_std2},
    title="Example Multi-Dataset Report", verbosity=1
).write_html("../tutorial_files/exampleMultiDataSetReport", auto_open=True, verbosity=1)

--- Circuit Creation ---
   616 sequences created
   Dataset has 1456 entries: 616 utilized, 0 requested sequences were missing
-- Std Practice:  Iter 1 of 3  (TP) --: 
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.244829997162508
  1.1936677889884049
  0.9868539533169904
  0.9321977240915884
  0.04714742318656958
  0.012700520808584755
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119284
  1.4142135623730956
  1.4142135623730954
  1.4142135623730951
  2.5394445830714747e-16
  1.118988490269554e-16
  
  --- Iterative MLGST: Iter 1 of 5  92 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 50.2568 (92 data params - 31 model params = expected mean of 61; p-value = 0.835246)
    Completed in 0.3s
    2*Delta(log(L)) = 50.4026
    Iteration 1 took 0.3s
    
  --- Iterative MLGST: Iter 2 of 5  168 operation sequences ---: 
    --- Minimum Chi^2 GST ---
    Sum of Chi^2 = 112.85 (168 data params - 

/Users/enielse/pyGSTi/pygsti/report/factory.py:1250: UserWarning:

Not all data sets are comparable - no comparisions will be made.

/Users/enielse/pyGSTi/pygsti/report/factory.py:1303: UserWarning:

PDF output is not available for reports with multiple datasets



### Other cool `Report` tricks
Finally, let us highlight a few of the additional arguments one can supply to the respective `Report` output methods that allows further control over what gets included in the generated report.

- Setting the `link_to` argument to a tuple of `'pkl'`, `'tex'`, and/or `'pdf'` will create hyperlinks within the plots or below the tables of the HTML linking to Python pickle, LaTeX source, and PDF versions of the content, respectively.  The Python pickle files for tables contain pickled pandas `DataFrame` objects, wheras those of plots contain ordinary Python dictionaries of the data that is plotted.  Applies to HTML reports only.

- Setting the `brevity` argument to an integer higher than $0$ (the default) will reduce the amount of information included in the report (for details on what is included for each value, see the doc string).  Using `brevity > 0` will reduce the time required to create, and later load, the report, as well as the output file/folder size.  This applies to both HTML and PDF reports.

Below, we demonstrate both of these options in very brief (`brevity=4`) report with links to pickle and PDF files.  Note that to generate the PDF files you must have `pdflatex` installed.

In [11]:
pygsti.report.construct_standard_report(
    results_std, title="Example Brief Report", verbosity=1
).write_html("../tutorial_files/exampleBriefReport", auto_open=True, verbosity=1,
             brevity=4, link_to=('pkl', 'pdf'))

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI
Found standard clifford compilation from smq1Q_XYI


/Users/enielse/pyGSTi/pygsti/report/factory.py:1308: UserWarning:

PDF output is not available for reports with multiple gauge opts



## Advanced Reports: `Report.write_notebook`
In addition to the standard HTML-page reports demonstrated above, pyGSTi is able to generate a Jupyter notebook containing the Python commands to create the figures and tables within a general report.  This is facilitated
by `Workspace` objects, which are factories for figures and tables (see previous tutorials).  By calling `Report.write_notebook`, all of the relevant `Workspace` initialization and calls are dumped to a new notebook file, which can be run (either fully or partially) by the user at their convenience.  Creating such "report notebooks" has the advantage that the user may insert Python code amidst the figure and table generation calls to inspect or modify what is display in a highly customizable fashion.  The chief disadvantages of report notebooks is that they require the user to 1) have a Jupyter server up and running and 2) to run the notebook before any figures are displayed.

The line below demonstrates how to create a report notebook using `write_notebook`.  Note that the argument list is very similar to the other `Report` output methods.

In [12]:
pygsti.report.construct_standard_report(
    results, title="GST Example Report Notebook", confidenceLevel=None, verbosity=3
).write_notebook("../tutorial_files/exampleReport.ipynb", auto_open=True, connected=False, verbosity=3)

Running idle tomography
Computing switchable properties
Found standard clifford compilation from smq1Q_XYI
Report Notebook created as ../tutorial_files/exampleReport.ipynb


## Multi-qubit reports
The dimension of the density matrix space with with more than 2 qubits starts to become quite large, and Models for 3+ qubits rarely allow every element of the operation process matrices to vary independently.  As such, many of the figures generated by `construct_standard_report` are both too unwieldy (displaying a $64 \times 64$ grid of colored boxes for each operation) and not very helpful (you don't often care about what each element of an operation matrix is).  For this purpose, we are developing a report that doesn't just dump out and analyze operation matrices as a whole, but looks at a `Model`'s structure to determine how best to report quantities.  This "n-qubit report" is invoked using `pygsti.report.construct_nqnoise_report`, and has similar arguments to `construct_standard_report`.  It is, however <b style="color:red">still under development</b>, and while you're welcome to try it out, it may crash or not work in other weird ways.